In [ ]:
#test if subsetting object and using sc.pp.calculate_qc_metrics yields same results (yes it does)

#anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
#anndataobject.obs["new_annotation"]
#neural = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#megakaryocyte = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#msc = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#fibroblast = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#myeloid = anndataobject[anndataobject.obs.new_annotation == "Neural"]

#test_neural = sc.pp.calculate_qc_metrics(neural, inplace=True)
#neural.var.to_csv("test_neural")

In [ ]:
#test to check data of individual genes to compare them later to the seurat data
#anndataobject[:,"ISG15"].X.todense().sum()
#sub_object.uns["Average_Expression"]

In [ ]:
#how to add tf activities as a new anndata object with the same obs/metadata

anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")

tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs
tf_activities.X
sc.pp.scale(tf_activities)
tf_activities.X


if isinstance(arguments_list["tf_activities"], str):
        tf_activities = ad.read_csv(arguments_list["tf_activities"])
        tf_activities.obsm = anndataobject.obsm
        tf_activities.uns = anndataobject.uns
        tf_activities.obs = anndataobject.obs

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    z_scores = []  # Initialize an empty list to store the z-scores

    # Iterate over each row in anndataobject_markers
    for idx, row in anndataobject_markers.iterrows():
        gene = row["gene"]  # Get the gene name
        cluster = row["cluster"]  # Get the corresponding cluster

        # Check if the gene exists in the filtered dataframe
        if gene in filtered_summarized_tf_scores_df.index:
            # Check if the cluster exists as a column in filtered_summarized_tf_scores_df
            if cluster in filtered_summarized_tf_scores_df.columns:
                # Append the z-score for the gene and cluster
                z_scores.append(filtered_summarized_tf_scores_df.loc[gene, cluster])
            else:
                # If the cluster is not found, append a placeholder value (e.g., None or np.nan)
                z_scores.append(None)
        else:
            # If the gene is not found, append a placeholder value (e.g., None or np.nan)
            z_scores.append(None)

    # Return the full list of z-scores
    return z_scores


In [ ]:
#not per cluster but cluster and pval etc need to be added to csv (check against specific marker csv from lr2tf test run in R)

def get_significant_tfs_single(sub_object, celltype, condition, out_path, pval, logfc):
    
    #does not work if condition is None
    single_result_path = out_path + condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)
    
    #sc.pp.scale(tf_activities)
    #or sc.pp.normalize_total(anndataobject)
    # sc.pp.log1p(anndata_object) 
    

    number_of_clusters = len(sub_object.obs[celltype].cat.categories) 
    
    #//TODO: 
    sc.tl.rank_genes_groups(sub_object.obsm["ulm_estimate"].X, groupby= celltype, reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
     
    sc.tl.rank_genes_groups(sub_object.obsm["ulm_estimate"].X, groupby= celltype, reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

    #FindAllMarkers(seuratobject, only.pos = TRUE, min.pct = 0, logfc.threshold = 0, verbose = FALSE)

    result1 = sub_object.obsm["ulm_estimate"].X.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = sub_object.obsm["ulm_estimate"].X.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(sub_object.obsm["ulm_estimate"].X, group = None, log2fc_min=0, key="wilcoxon_markers")
    anndataobject_markers_wilcoxon.to_csv("specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(sub_object.obsm["ulm_estimate"].X, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
    anndataobject_markers_t_over.to_csv("specific_markers_t_test_overestim_test.csv",index=0)

    

    res = list()

    return res

In [ ]:
#cell is only for trying out stuff
#anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")

#tf_activities = ad.read_csv("decoupler_results.csv")
#tf_activities.obsm = anndataobject.obsm
#tf_activities.uns = anndataobject.uns
#tf_activities.obs = anndataobject.obs

#sc.pp.scale(tf_activities,zero_center=True)
#del tf_activities.var["mean"]
#del tf_activities.var["std"]
#or sc.pp.normalize_total(tf_activities)

#expects logarithmized data
#sc.tl.rank_genes_groups(tf_activities, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
#sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
     
#sc.tl.rank_genes_groups(tf_activities, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers")
#sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

#FindAllMarkers(seuratobject, only.pos = TRUE, min.pct = 0, logfc.threshold = 0, verbose = FALSE)

#result1 = tf_activities.uns['wilcoxon_markers']
#groups = result1['names'].dtype.names
#anndataobject_markers = pd.DataFrame(
#{group + '_' + key[:1]: result1[key][group]
#for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

#result2 = tf_activities.uns['t_test_overestim_var_markers']
#groups = result2['names'].dtype.names
#anndataobject_markers = pd.DataFrame(
#{group + '_' + key[:1]: result2[key][group]
#for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

#anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities, group = None, log2fc_min=0, key="wilcoxon_markers")
#anndataobject_markers_wilcoxon.to_csv("specific_markers_wilcoxon_test.csv",index=0)

#anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
#anndataobject_markers_t_over.to_csv("specific_markers_t_test_overestim_test.csv",index=0)
    

In [14]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os

tf_activities = ad.read_csv("decoupler_results.csv")
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
anndataobject.obsm['ulm_estimate'] = tf_activities 

anndataobject.obsm['ulm_estimate'].X

array([[-2.0571237 , -1.7287707 , -0.650971  , ..., -0.0064658 ,
        -1.5051594 ,  0.94431335],
       [-1.504947  , -1.086288  , -0.07632519, ..., -0.35059482,
        -0.91545016, -0.4751749 ],
       [-0.88697547,  1.0191329 , -0.35802716, ...,  0.8647811 ,
         0.1425965 ,  0.70262975],
       ...,
       [-1.1096482 , -1.4124041 , -1.080394  , ...,  0.8168695 ,
         0.15119185,  0.00681896],
       [-1.0042653 , -0.8440094 , -1.2228961 , ...,  0.44726205,
        -0.9731916 , -0.63016224],
       [-1.2844242 , -1.0794511 , -1.3090532 , ..., -0.03914427,
        -1.2446797 ,  0.43016624]], dtype=float32)

In [ ]:
    #checks for tf activity csv, if nothing there, runs decoupler
    if isinstance(arguments_list["tf_activities"], str):
        tf_activities = ad.read_csv(arguments_list["tf_activities"])
        anndataobject.obsm['ulm_estimate'] = tf_activities 
    elif arguments_list["tf_activities"] is None:
         dc.run_ulm(mat = anndataobject, net = "reg", source ='source', target ='target', weight ='weight', verbose = True, use_raw = False)
         tf_activities = anndataobject.obsm['ulm_estimate']
         tf_activities.to_csv("decoupler_results.csv")